In [ ]:
%pip install -Uqq fastbook nbdev
from fastbook import *

In [ ]:
urls = search_images_ddg('pug', max_images=100) # DuckDuckGo, can search any image
len(urls), urls[0]

In [ ]:
dest = Path('images/pug.jpg')
if not dest.exists(): download_url(urls[0], dest, show_progress=True)
im = Image.open(dest)
im.thumbnail((256,256))
im

In [ ]:
path = Path('images/pug-or-not')

if not path.exists():
    path.mkdir()

searches = [
    'pug',
    'french bulldog',
    'chihuahua',
]

for item in searches:
    if not (path/item).exists():
        print(f"Downloading {item} images")
        dest = (path/item)
        dest.mkdir(parents=True, exist_ok=True)
        results = search_images_ddg(f'{item} dog')
        download_images(dest, urls=results[:200])
        resize_images(dest, max_size=400, dest=dest)

In [ ]:
# Remove broken images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink);
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # ImageBlock: input, CategoryBlock: output
    get_items=get_image_files, # get images from folder
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # split into train/valid sets (20% validation) and seed for reproducibility
    get_y=parent_label, # get label from parent folder
    item_tfms=[Resize(192, ResizeMethod.Pad, pad_mode='zeros')]
).dataloaders(path)

dls.show_batch(max_n=12, nrows=3)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(8)

In [ ]:
interpretation = ClassificationInterpretation.from_learner(learn)
interpretation.plot_confusion_matrix()

In [ ]:
interpretation.plot_top_losses(6, nrows=3, figsize=(12, 8))

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
deleted_files = []
for idx in cleaner.delete():
    try:
        deleted_files.append(cleaner.fns[idx].unlink())
    except Exception as e:
        print(f"Error deleting {cleaner.fns[idx]}: {e}")
print(f"Deleted {len(deleted_files)} files")

moved_files = []
for idx,cat in cleaner.change():
    moved_files.append(shutil.move(str(cleaner.fns[idx]), path/cat))
print(f"Moved {len(moved_files)} files")

In [ ]:
searches.sort()
categories = searches
print(categories)

In [ ]:
predicted_dog,_,probs = learn.predict(PILImage.create('images/chihuahua.jpg'))


cats_and_probs = {cat: probs[i].item() for i, cat in enumerate(categories)}
print(cats_and_probs)

print(f"This is a: {predicted_dog}.")

dog = 'chihuahua'
print(f"Probability it's a {dog}: {cats_and_probs[dog]:.6f}")

Now we export our Learner model.

In [ ]:
learn.export('pug_or_not.pkl')

Now the model can be put into production and used to make predictions on new data.

This [HuggingFace space](https://huggingface.co/spaces/abramprz/image-classifier) uses the model.